In [13]:
import requests
import visualisationGlock as v
import json
import matplotlib as plt
from IPython.display import clear_output

PORT=8000
sim_id = "sim1"
url = f'http://127.0.0.1:{PORT}/control/poll/create_from_example/{sim_id}'
response = requests.post(url)
print(response)

<Response [201]>


In [24]:
url = f'http://127.0.0.1:{PORT}/random_number?count=40'
response = requests.get(url, stream=True)
t = 0
graph = v.LineGraph("Funds in euro", x_legend='Hours', y_legend="Funds in euro")
for line in response.iter_lines():
    if line:
        t += 1
        decoded_line = line.decode('utf-8')
        jresponse = json.loads(decoded_line)
        
        x = jresponse["funds_in_eur"]

        graph.addPoint(t,x)

TypeError: __init__() got an unexpected keyword argument 'x'